In [26]:
import cirq
import numpy as np
from mitiq.benchmarks import generate_rb_circuits
from mitiq import MeasurementResult, Observable, PauliString
from functools import partial

n_qubits = 2

circuit = generate_rb_circuits(2, 10)[0]
# a, b = cirq.LineQubit.range(2)
obs = Observable(PauliString("ZZ"))
# circuit = cirq.Circuit(
#     cirq.H(a), cirq.CNOT(a, b), cirq.measure(a, b)
# )

def execute(circuit: cirq.Circuit, noise_level: float = 0.002, p0: float = 0.05) -> MeasurementResult:
    """Execute a circuit with depolarizing noise of strength ``noise_level`` and readout errors ...
    """
    measurements = circuit[-1]
    circuit =  circuit[:-1]
    circuit = circuit.with_noise(cirq.depolarize(noise_level))
    circuit.append(cirq.bit_flip(p0).on_each(circuit.all_qubits()))
    circuit.append(measurements)

    simulator = cirq.DensityMatrixSimulator()

    result = simulator.run(circuit, repetitions=1000)
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings)


# REM
from mitiq import rem

p0 = p1 = 0.05
icm = rem.generate_inverse_confusion_matrix(2, p0, p1)
rem_executor = rem.mitigate_executor(execute, inverse_confusion_matrix=icm)

rem_result = obs.expectation(circuit, rem_executor)
print("Mitigated value obtained with REM:", "{:.5f}".format(rem_result.real))


# Stack ZNE on top of REM
from mitiq import zne
rem_zne_executor = zne.mitigate_executor(rem_executor, observable=obs, scale_noise=zne.scaling.folding.fold_global)

rem_zne_result = rem_zne_executor(circuit)
print("Mitigated value obtained with REM + ZNE:", "{:.5f}".format(rem_zne_result.real))

# stack REM + DDD 
from mitiq import ddd
rule = ddd.rules.xx

rem_ddd_executor = ddd.mitigate_executor(
    rem_executor, rule=rule, observable=obs
)

rem_ddd_result = rem_ddd_executor(circuit)
print("Mitigated value obtained with REM + DDD:", "{:.5f}".format(rem_ddd_result.real))

# stack DDD on top of that
from mitiq import ddd
rule = ddd.rules.xx

rem_zne_ddd_executor = ddd.mitigate_executor(
    rem_zne_executor, rule=rule
)

rem_zne_ddd_result = rem_zne_ddd_executor(circuit)
print("Mitigated value obtained with REM + ZNE + DDD:", "{:.5f}".format(rem_zne_ddd_result.real))

Mitigated value obtained with REM: 0.75800
Mitigated value obtained with REM + ZNE: 0.99800
Mitigated value obtained with REM + DDD: 0.72400


AttributeError: 'NoneType' object has no attribute '_expectation_from_measurements'

In [27]:
zne.execute_with_zne(circuit, execute, obs)

(0.9700000000000004+0j)